<a href="https://colab.research.google.com/github/LeonardoDipilato/Yomi-MU-Data/blob/main/Yomi_MU_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Yomi MU Analysis
**Yomi** is a 2-players card game by *David Sirlin* in which both players choose a character to play between 20 pre-made character decks.

Although considered a very-well balanced game, when two characters face one has inevitably the advantage over the other (considering it "balanced" means that few to no matchups give a large advantage to one player). This is called a positive Match-Up, shortened to MU, while being a negative MU for the other character.

As such, some characters may be better overall, and others might be worse overall. Determining the value of each character lets us create what's called a Tier List, which is important for this kind of games since it lets people know which characters are more relevant and thus which characters you should know how to play (or play against) as better as possible.

## Setup

In [1]:
import requests
import numpy as np
import pandas as pd

MUCs are hard to find in a methodical way, since the player's skill levels also influence the outcome of some matches.

Usually, MUCs are either generated from tournament data, if there's an extensive number of games for each MU, or from player's experience who estimate the advantage a character has against another character.

There are a lot of MUCs created by different players in the Yomi community. One of those is **Scymrian's Composite MUC**, which is computed as the average of the other MUCs.

The one we'll be using is an adjusted version of Scymrian's MUC, which we'll store in a variable called, aptly, `MUC`.

In [2]:
mucURL = "https://raw.githubusercontent.com/LeonardoDipilato/Yomi-MU-Data/main/data/ScymrianMUC.csv"
namesURL = "https://raw.githubusercontent.com/LeonardoDipilato/Yomi-MU-Data/main/data/names.txt"
reqMUC = requests.get(mucURL)
reqNames = requests.get(namesURL)

MUC = np.array([[float(v)/10 for v in row.split("\t")] for row in reqMUC.text.split("\n")])
names = reqNames.text.split("\n")

MUCdf = pd.DataFrame(MUC, index=names, columns=names)
def dfToPercent(df):
  return df.applymap(lambda x: f"{round(x*100, 2)}%")
dfToPercent(MUCdf)


,Grave,Jaina,Midori,Setsuki,Rook,Degrey,Valerie,Geiger,Lum,Argagarg,Quince,Onimaru,Troq,Bal-Bas-Beta,Menelker,Persephone,Gloria,Gwen,Vendetta,Zane
Grave,50.0%,56.25%,52.19%,54.37%,52.19%,44.38%,57.81%,54.06%,56.56%,40.0%,46.56%,51.25%,50.0%,60.62%,50.31%,52.19%,49.69%,56.88%,58.13%,54.69%
Jaina,43.75%,50.0%,50.31%,40.0%,53.12%,46.25%,52.19%,42.5%,43.75%,40.94%,55.94%,44.69%,51.88%,59.38%,44.38%,52.19%,47.5%,50.62%,49.38%,45.62%
Midori,47.81%,49.69%,50.0%,55.0%,44.38%,40.62%,50.0%,35.62%,43.75%,52.5%,41.88%,42.81%,49.38%,35.31%,55.62%,49.38%,53.75%,54.69%,52.5%,42.81%
Setsuki,45.62%,60.0%,45.0%,50.0%,51.88%,53.75%,50.0%,54.69%,45.94%,53.44%,48.75%,45.31%,41.88%,58.13%,53.75%,56.56%,58.75%,53.12%,55.62%,47.81%
Rook,47.81%,46.88%,55.62%,48.12%,50.0%,48.75%,44.38%,39.06%,40.0%,53.44%,48.12%,43.75%,48.12%,37.19%,53.75%,46.56%,52.5%,49.06%,44.38%,55.62%
Degrey,55.62%,53.75%,59.38%,46.25%,51.25%,50.0%,56.25%,51.88%,59.06%,51.88%,47.5%,65.62%,45.0%,55.62%,55.0%,46.25%,59.38%,58.75%,53.44%,43.44%
Valerie,42.19%,47.81%,50.0%,50.0%,55.62%,43.75%,50.0%,48.12%,44.69%,50.62%,58.13%,43.12%,45.0%,60.0%,55.31%,56.25%,53.12%,60.0%,62.5%,49.69%
Geiger,45.94%,57.5%,64.38%,45.31%,60.94%,48.12%,51.88%,50.0%,51.25%,52.81%,65.0%,48.12%,59.06%,53.44%,42.81%,40.62%,45.0%,46.25%,51.56%,44.38%
Lum,43.44%,56.25%,56.25%,54.06%,60.0%,40.94%,55.31%,48.75%,50.0%,35.0%,53.12%,54.37%,58.75%,60.0%,45.0%,47.19%,58.13%,55.94%,62.5%,50.31%
Argagarg,60.0%,59.06%,47.5%,46.56%,46.56%,48.12%,49.38%,47.19%,65.0%,50.0%,47.19%,65.0%,38.12%,59.38%,51.88%,43.44%,35.62%,56.25%,46.88%,42.5%


Another important information is the current state of the metagame. If you don't have this information, you can estimate it, but luckily we do have the number of games played in the last years (since 2014) in tournaments with each character.

We call the current state of the metagame `step0`.

In terms of game theory, the current metagame can also be called ***strategy***, because it expresses how often your opponent will choose each option.



In [3]:
histDataURL = "https://raw.githubusercontent.com/LeonardoDipilato/Yomi-MU-Data/main/data/histdata.txt"
reqHist = requests.get(histDataURL)

step0 = np.array([float(v) for v in reqHist.text.split("\n")])
step0 = step0/sum(step0)

##Finding out the best character to play
If you want to find out which character has the best odds against a specific strategy, you can use the MUC to find out which characters have a good overall chance of success.

One way to do that is to consider every single character you could choose and see how it fares against every character, weighting the results based on how popular that character (if a character is not played a lot we don't care much about how well we fare against that character, while we do care about more popular characters since we'll get to face them more often) is.

If we call the MUC as the matrix $M$, the strategy the opponent is using as the vector $\vec{s}$, we can get the apriori winning probability $\vec{p}$ (i.e. the probability of winning with a certain character knowing the opponent's strategy, but not the character they'll actually use) by simply calculating:

$$\vec{p} = M \cdot \vec{s}$$

With this, we can simply get the highest value and know which character has the best probability of winning a game given that specific strategy.

In [12]:
p = np.dot(MUC, step0)
dfToPercent(pd.DataFrame(p, names, ["Apriori winning %"]).sort_values(by='Apriori winning %', ascending=False))

,Apriori winning %
Troq,52.65%
Degrey,52.46%
Zane,51.94%
Grave,51.91%
Lum,51.69%
Geiger,50.99%
Setsuki,50.81%
Valerie,50.63%
Onimaru,50.62%
Argagarg,49.6%


We can see, then, that picking **Troq** will win you 52.65% of the games you'll play against the current metagame.

We can do a couple more things with this approach in mind. Let's assume, for example, that after this Tier List is published, people decide to change their main character according to it. Most specifically, the stronger the character, the more likely they are to pick that character.

In mathematical terms, we can write:

$$\vec{s_1} = \frac{\vec{p}}{||\vec{p}||_1} = M \cdot \vec{s_0}$$

Where $\vec{s_0}$ is the strategy from before the tier list was released, $\vec{s_1}$ is the new strategy and $||\vec{p}||_1$ is the *1-norm* of $\vec{p}$, which basically is the sum of all its components.

In [30]:
step1 = p/sum(p)
dfToPercent(pd.DataFrame(step1, names, ["Coverage %"]).sort_values(by='Coverage %', ascending=False))

,Coverage %
Troq,5.3%
Degrey,5.28%
Zane,5.23%
Grave,5.23%
Lum,5.2%
Geiger,5.13%
Setsuki,5.11%
Valerie,5.1%
Onimaru,5.1%
Argagarg,4.99%


Since the metagame changed a little, we can recompute the Tier List with this updated values, generating $\vec{s_2}$.

If we repeat this process multiple times, eventually we will get to a state $\vec{s_\infty}$ for which applying this process again and again doesn't change the metagame. This is an equilibrium state for our problem.

In algebra, the vector $\vec{s}$ such that $\vec{s} = M \cdot \vec{s}$ is called principal eigenvector of the matrix $M$.

In [47]:
tol = 1e-8
oldstep = np.zeros(20)
sol = step0
while max(abs(sol-oldstep)) > tol:
  oldstep = sol
  sol = np.dot(MUC, oldstep)
  sol = sol/sum(sol)

dfToPercent(pd.DataFrame(sol, index=names, columns=['Coverage %']).sort_values(by='Coverage %', ascending=False))

,Coverage %
Degrey,5.33%
Zane,5.27%
Grave,5.24%
Lum,5.22%
Troq,5.21%
Setsuki,5.15%
Valerie,5.12%
Geiger,5.12%
Onimaru,5.07%
Argagarg,5.04%


Finally, another thing we might be interested in is a strategy $\vec{S}$ that cannot be beaten.

Most specifically, this means that no character has an expected apriori winning probability against $\vec{S}$ higher than $50\%$.

By extension this also means

In mathematical terms, we can write:

$$\max(M \cdot \vec{S}) < 0.5$$



In [41]:
from scipy.optimize import minimize, LinearConstraint

def optimize(MUC, step0):
  #The function to minimize is:
  def func(a):
    ap = np.zeros(20)
    ap[:-1] = a
    ap[-1] = 1-sum(a)
    return max(np.dot(MUC,ap))

  #the strategy vector s has 20 options, but the last one can be expressed as 1-sum(s[:-1])
  a0 = step0[:-1]

  #each component cannot be smaller than 0 and larger than 1
  #This lets me write 19 double-sided inequalities
  #Also, one final restraint is that the sum of all 19 explicit options cannot equal more than 1

  lcMatrix = np.zeros((20, 19))
  lcMatrix[:-1, :]=np.eye(19,19)
  lcMatrix[-1, :]=np.ones(19)

  #The left constraints are 0s everywhere
  left = np.zeros(20)

  #The right constraints are 1s everywhere
  right = np.ones(20)

  linear_constraint = LinearConstraint(lcMatrix, left, right)

  sol = minimize(func, a0, constraints=linear_constraint).x
  eq = np.zeros(20)
  eq[:-1] = sol
  eq[-1] = 1-sum(sol)
  return dfToPercent(pd.DataFrame(eq, index=names, columns=["Coverage %"]).sort_values(by="Coverage %", ascending=False))

optimize(MUC, step0)

,Coverage %
Troq,25.8%
Grave,22.58%
Degrey,22.25%
Persephone,12.07%
Geiger,11.52%
Bal-Bas-Beta,5.69%
Onimaru,0.09%
Rook,0.0%
Gloria,0.0%
Midori,0.0%


## Introducing Matches
What we simulated until now was how likely a character was to win against a strategy. On the other hand, real matches of Yomi are played as a Best of 5 (Bo5). A Bo5 match has the following rules:

*   For the first game, players pick their character in a double-blind fashion;
*   From the second game onwards, the player who lost the game can pick a new character (this is called counterpick), while the winner has to stick with their character;
* First to 3 wins wins the match.

This is used to avoid a very unlucky first game in which a player, out of sheer luck, chooses a character that has a very high likelyhood of winning against the opponent's character, making for a large advantage.

BoN (Best-of N, first to $(N+1)/2$ wins) has the effect to smooth out the effects of the first pick, giving the edge to players who are good at many characters and know the game overall better. However, how much is this smoothing effect?

### Bo1
The simplest case of BoN is Bo1, which is a single game. In Bo1, there's only one game being played, so there's no counterpicking.

The likelihood of winning a match in Bo1 is the likelihood of winning a single game with the character you decide to play against the opponent's strategy. Thus, if the strategy the opponent is using is the one from historical data (`step0`), the best character you can pick is the top character in `step1` (which happens to be Troq).

###Bo3
Bo3 is first real BoN case, which will be the base for our calculations.

Let's start from a simple scenario: I choose to play Grave (numbered 1), and my opponent chooses to play Jaina (numbered 2. After seeing this, I want to find out how likely I am to win the match.

The probability of winning the match is the sum of three scenarios:

*   I win this game and win the next one
*   I win this game, lose the next, and win the last one
*   I lose this game and win the following two games

We can write this as:
*   I win this game and win at one game before my opponent wins two
*   I lose this game and win two games before my opponent wins one

The probability $P$ of winning is

$$P(2, 2, 0, 1)=M_{01} \cdot (1-P_c(2,1,0)) + (1-M_{01}) \cdot P_c(2,1,1)$$
Where:
* $P(n,m,i,j)$ is the probability of winning a match in which player 1 needs $n$ victories to win while player 2 needs $m$ considering that in the current game player 1 picked character $i$ and player 2 picked character $j$.
* $P_c(n,m,i)$ is the probability that the player who does the counterpicking wins a match in which they need $n$, the opponent needs $m$ and the character that's being counterpicked is character $i$

$P_c(n,m,i)$ can be computed in a very similar way, since you need to compute $P(n,m,i,j)$ for every $j$ and just choose the best option.

Repeating this process is called *recursion*, and it's a very powerful way to do math and programming.

The problem about recursion is that if you stop there, the program will never finish running. That's it, unless you give a base case to the recursion, in which case $P$ can be calculated without calling itself again and again.

The base case we'll use is that $P(0, m, i, j)=1$ and $P(n, 0, i, j)=0$ for any $n$,$m$,$i$,$j$.

We can then write a function `counterpick` that given the current state of the match tells you the most optimal character you can pick.

In [4]:
computedStates = {}

def counterpick(yourScore, oppScore, oppCharacter, M=MUC):
  global computedStates
  def formattedState():
    return f"{yourScore}:{oppScore};{oppCharacter}"
  
  state = formattedState()
  if state in computedStates:
    return computedStates[state]
  maxFunc = lambda v: np.array([np.argmax(v), max(v)])
  if (yourScore == 0):
    return (-1, 1)
  if (oppScore == 0):
    return (-1, 0)
  res = maxFunc(M[:, oppCharacter]*[1-counterpick(oppScore, yourScore-1, i)[1] for i in range(20)] + \
                  (1-M[:, oppCharacter])*[counterpick(yourScore, oppScore-1, i)[1] for i in range(20)])
  computedStates[state] = res
  return res

Using the `counterpick` function we can finally try and compute a MUC for Bo3. Most specifically, if we simulate every possible set of starting characters for both players and compute the likelihood of player 1 winning that game we can build a matrix that works the same as the MUC we've been using until now, but instead of telling you how likely you are to win a game it tells you how likely you are to win the match.

In [73]:
def boNmuc(BO, M=MUC):
  mboN = np.zeros((20, 20))
  scores = (BO+1)/2
  for char in range(20):
      for opponent in range(char, 20):
          MU = M[char, opponent]*(1-counterpick(scores, scores-1, char)[1]) + (1-M[char, opponent])*counterpick(scores, scores-1, opponent)[1]
          mboN[char, opponent] = MU
          mboN[opponent, char] = 1-MU
  return mboN

MUC3 = boNmuc(3)
dfToPercent(pd.DataFrame(MUC3, index=names, columns=names))

,Grave,Jaina,Midori,Setsuki,Rook,Degrey,Valerie,Geiger,Lum,Argagarg,Quince,Onimaru,Troq,Bal-Bas-Beta,Menelker,Persephone,Gloria,Gwen,Vendetta,Zane
Grave,50.0%,53.65%,52.08%,52.09%,51.41%,46.18%,54.15%,52.16%,54.04%,45.53%,49.42%,52.76%,49.92%,55.94%,50.03%,51.35%,50.66%,54.14%,55.12%,52.84%
Jaina,46.35%,50.0%,50.67%,44.23%,51.38%,46.73%,50.76%,45.8%,47.19%,45.61%,53.52%,49.23%,50.39%,54.74%,46.51%,50.85%,49.11%,50.51%,50.32%,47.8%
Midori,47.92%,49.33%,50.0%,51.32%,46.54%,43.37%,49.14%,41.95%,46.74%,50.8%,46.27%,47.92%,48.61%,42.45%,51.68%,48.93%,51.62%,51.95%,51.29%,45.96%
Setsuki,47.91%,55.77%,48.68%,50.0%,51.45%,51.37%,50.32%,52.69%,48.88%,52.52%,50.68%,50.12%,45.86%,54.89%,52.01%,53.8%,55.41%,52.45%,54.09%,49.52%
Rook,48.59%,48.62%,53.46%,48.55%,50.0%,48.22%,46.98%,44.2%,45.49%,52.0%,49.91%,48.95%,48.66%,43.88%,51.49%,48.19%,51.75%,49.92%,48.06%,52.98%
Degrey,53.82%,53.27%,56.63%,48.63%,51.78%,50.0%,54.27%,51.89%,56.27%,52.39%,50.67%,60.65%,48.06%,54.33%,53.37%,49.07%,56.48%,56.05%,53.68%,47.84%
Valerie,45.85%,49.24%,50.86%,49.68%,53.02%,45.73%,50.0%,48.97%,47.96%,50.76%,54.98%,48.8%,47.2%,55.46%,52.47%,53.28%,52.23%,55.55%,57.12%,50.16%
Geiger,47.84%,54.2%,58.05%,47.31%,55.8%,48.11%,51.03%,50.0%,51.3%,51.93%,58.44%,51.22%,54.53%,52.24%,46.07%,45.37%,48.25%,48.73%,51.8%,47.54%
Lum,45.96%,52.81%,53.26%,51.12%,54.51%,43.73%,52.04%,48.7%,50.0%,42.49%,51.87%,53.43%,53.58%,54.73%,46.59%,48.09%,54.01%,52.83%,56.35%,49.86%
Argagarg,54.47%,54.39%,49.2%,47.48%,48.0%,47.61%,49.24%,48.07%,57.51%,50.0%,49.18%,58.51%,43.31%,54.62%,50.23%,46.36%,43.2%,53.17%,49.01%,46.15%


If we compare the two MUCs we can see that the Bo3 MUs are less extreme than the Bo1 MUs (which means that the results are less skewed by a suboptimal first game pick).

In [80]:
n1 = np.argmax(MUC)
i1 = n//20
j1 = n%20
print(f"The best MU in Bo1 is {names[i1]} vs. {names[j1]} with a {round(MUC[int(n/20), n%20]*100, 2)}% probability of winning")

n3 = np.argmax(MUC3)
i3 = n//20
j3 = n%20
print(f"The best MU in Bo3 is {names[i3]} vs. {names[j3]} with a {round(MUC3[i3, j3]*100, 2)}% probability of winning")

print(f"The root mean squared difference between the Bo1 MUC and the Bo3 MUC is {np.sqrt(np.mean(((MUC-MUC3)**2)))}")

The best MU in Bo1 is Degrey vs. Onimaru with a 65.62% probability of winning
The best MU in Bo3 is Degrey vs. Onimaru with a 60.65% probability of winning
The root mean squared difference between the Bo1 MUC and the Bo3 MUC is 3.1382090015802215%


###Bo5
The reasoning we used for Bo3 applies to any BoN, so computing the new Bo5 matrix is as easy as changing the $3$ to a $5$ in the code.

In [77]:
MUC5 = boNmuc(5)
dfToPercent(pd.DataFrame(MUC5, index=names, columns=names))

,Grave,Jaina,Midori,Setsuki,Rook,Degrey,Valerie,Geiger,Lum,Argagarg,Quince,Onimaru,Troq,Bal-Bas-Beta,Menelker,Persephone,Gloria,Gwen,Vendetta,Zane
Grave,50.0%,53.02%,52.16%,52.01%,51.7%,47.37%,53.27%,51.95%,53.35%,47.19%,50.15%,52.38%,50.38%,54.95%,50.16%,51.69%,50.6%,53.48%,54.36%,52.64%
Jaina,46.98%,50.0%,50.89%,45.8%,51.47%,47.63%,50.43%,46.86%,47.86%,47.1%,53.06%,49.47%,50.54%,53.8%,47.26%,51.11%,49.21%,50.49%,50.48%,48.61%
Midori,47.84%,49.11%,50.0%,50.79%,47.47%,44.74%,48.79%,43.64%,47.18%,50.6%,47.27%,48.13%,48.81%,44.14%,50.82%,49.28%,50.75%,51.17%,50.81%,46.88%
Setsuki,47.99%,54.2%,49.21%,50.0%,51.31%,50.96%,49.87%,51.92%,48.97%,52.09%,50.73%,49.96%,46.93%,53.68%,51.27%,53.1%,53.85%,51.75%,53.12%,49.72%
Rook,48.3%,48.53%,52.53%,48.69%,50.0%,48.35%,47.11%,45.31%,46.21%,51.46%,49.93%,48.88%,48.82%,45.18%,50.64%,48.72%,50.81%,49.61%,48.37%,52.07%
Degrey,52.63%,52.37%,55.26%,49.04%,51.65%,50.0%,52.99%,51.4%,54.69%,52.09%,50.8%,58.07%,48.66%,53.34%,52.4%,49.67%,54.76%,54.58%,52.9%,48.53%
Valerie,46.73%,49.57%,51.21%,50.13%,52.89%,47.01%,50.0%,49.46%,48.6%,51.16%,54.37%,49.28%,48.27%,54.55%,52.05%,53.13%,51.81%,54.55%,55.87%,50.57%
Geiger,48.05%,53.14%,56.36%,48.08%,54.69%,48.6%,50.54%,50.0%,50.93%,51.78%,56.67%,50.91%,53.59%,51.79%,46.83%,46.92%,48.46%,49.04%,51.52%,48.33%
Lum,46.65%,52.14%,52.82%,51.03%,53.79%,45.31%,51.4%,49.07%,50.0%,44.72%,51.79%,52.67%,52.94%,53.76%,47.3%,49.01%,52.99%,52.24%,55.04%,50.15%
Argagarg,52.81%,52.9%,49.4%,47.91%,48.54%,47.91%,48.84%,48.22%,55.28%,50.0%,49.4%,56.07%,44.85%,53.22%,49.71%,47.38%,44.35%,52.07%,49.1%,47.01%


Bo5 is more interesting on a practical level than Bo3 since the most basic tournament rules require a Bo5 to decide the winner of the match. We could then run a more in-depth analysis on Bo5.

In [93]:
print(f"The best MU in Bo1 is {names[i1]} vs. {names[j1]} with a {round(MUC[int(n/20), n%20]*100, 2)}% probability of winning")

n5 = np.argmax(MUC5)
i5 = n//20
j5 = n%20
print(f"The best MU in Bo5 is {names[i5]} vs. {names[j5]} with a {round(MUC5[i5, j5]*100, 2)}% probability of winning")

print(f"The root mean squared difference between the Bo1 MUC and the Bo5 MUC is {np.sqrt(np.mean(((MUC-MUC5)**2)))}")

The best MU in Bo1 is Degrey vs. Onimaru with a 65.62% probability of winning
The best MU in Bo5 is Degrey vs. Onimaru with a 58.07% probability of winning
The root mean squared difference between the Bo1 MUC and the Bo5 MUC is 0.03846132314908599


The Tier List at step 1 looks like this:

In [84]:
dfToPercent(pd.DataFrame(np.dot(MUC5, step0), index=names, columns=["Apriori winning %"]).sort_values(by="Apriori winning %", ascending=False))

,Apriori winning %
Degrey,51.93%
Grave,51.41%
Troq,51.32%
Valerie,50.76%
Setsuki,50.73%
Geiger,50.69%
Zane,50.58%
Lum,50.49%
Menelker,50.1%
Argagarg,49.46%


Its steady-state solution is:

In [85]:
tol = 1e-8
oldstep = np.zeros(20)
sol = step0
while max(abs(sol-oldstep)) > tol:
  oldstep = sol
  sol = np.dot(MUC5, oldstep)
  sol = sol/sum(sol)

dfToPercent(pd.DataFrame(sol, index=names, columns=['Coverage %']).sort_values(by='Coverage %', ascending=False))

,Coverage %
Degrey,5.23%
Grave,5.16%
Troq,5.11%
Valerie,5.1%
Setsuki,5.1%
Zane,5.09%
Geiger,5.08%
Lum,5.07%
Menelker,5.04%
Argagarg,4.98%


The strategy $S$ is:

In [86]:
optimize(MUC5, step0)

,Coverage %
Degrey,30.3%
Troq,26.98%
Geiger,20.08%
Persephone,9.35%
Grave,6.9%
Setsuki,6.39%
Quince,0.0%
Zane,0.0%
Rook,0.0%
Gloria,0.0%


###Bo7
Finally, despite not too interesting on a practical level, Bo7 shows how the first choice evolves with the number of games going up.

In [90]:
MUC7 = boNmuc(7)
dfToPercent(pd.DataFrame(MUC7, index=names, columns=names))

,Grave,Jaina,Midori,Setsuki,Rook,Degrey,Valerie,Geiger,Lum,Argagarg,Quince,Onimaru,Troq,Bal-Bas-Beta,Menelker,Persephone,Gloria,Gwen,Vendetta,Zane
Grave,50.0%,52.58%,51.83%,51.64%,51.47%,47.84%,52.66%,51.67%,52.85%,47.67%,50.15%,51.95%,50.3%,54.0%,50.08%,51.38%,50.51%,52.77%,53.51%,52.18%
Jaina,47.42%,50.0%,50.71%,46.4%,51.22%,47.99%,50.21%,47.38%,48.23%,47.54%,52.51%,49.46%,50.38%,52.97%,47.6%,50.83%,49.29%,50.19%,50.19%,48.76%
Midori,48.17%,49.29%,50.0%,50.6%,47.94%,45.63%,48.88%,44.73%,47.7%,50.49%,47.73%,48.38%,48.97%,44.89%,50.62%,49.35%,50.61%,50.81%,50.52%,47.35%
Setsuki,48.36%,53.6%,49.4%,50.0%,51.19%,50.87%,49.84%,51.69%,49.24%,51.8%,50.67%,49.97%,47.45%,52.98%,51.06%,52.61%,53.26%,51.36%,52.52%,49.78%
Rook,48.53%,48.78%,52.06%,48.81%,50.0%,48.59%,47.43%,46.09%,46.87%,51.17%,49.91%,48.97%,48.94%,45.74%,50.43%,48.84%,50.62%,49.45%,48.43%,51.64%
Degrey,52.16%,52.01%,54.37%,49.13%,51.41%,50.0%,52.39%,51.18%,53.93%,51.73%,50.67%,56.67%,48.85%,52.62%,51.93%,49.67%,53.95%,53.66%,52.26%,48.72%
Valerie,47.34%,49.79%,51.12%,50.16%,52.57%,47.61%,50.0%,49.68%,48.98%,51.07%,53.77%,49.44%,48.63%,53.78%,51.76%,52.69%,51.62%,53.78%,54.9%,50.54%
Geiger,48.33%,52.62%,55.27%,48.31%,53.91%,48.82%,50.32%,50.0%,50.8%,51.45%,55.53%,50.68%,52.94%,51.3%,47.27%,47.36%,48.69%,48.99%,51.08%,48.54%
Lum,47.15%,51.77%,52.3%,50.76%,53.13%,46.07%,51.02%,49.2%,50.0%,45.56%,51.45%,52.13%,52.37%,52.93%,47.63%,49.08%,52.43%,51.66%,54.02%,50.04%
Argagarg,52.33%,52.46%,49.51%,48.2%,48.83%,48.27%,48.93%,48.55%,54.44%,50.0%,49.52%,55.03%,45.68%,52.54%,49.7%,47.77%,45.29%,51.57%,49.08%,47.47%


In [91]:
print(f"The best MU in Bo1 is {names[i1]} vs. {names[j1]} with a {round(MUC[int(n/20), n%20]*100, 2)}% probability of winning")

n7 = np.argmax(MUC3)
i7 = n//20
j7 = n%20
print(f"The best MU in Bo7 is {names[i3]} vs. {names[j3]} with a {round(MUC7[i7, j7]*100, 2)}% probability of winning")

print(f"The root mean squared difference between the Bo1 MUC and the Bo7 MUC is {np.sqrt(np.mean(((MUC-MUC7)**2)))}")

The best MU in Bo1 is Degrey vs. Onimaru with a 65.62% probability of winning
The best MU in Bo7 is Degrey vs. Onimaru with a 56.67% probability of winning
The root mean squared difference between the Bo1 MUC and the Bo7 MUC is 0.04245375956938459


##Utilities
This document is interactive, which means that you can use all the code written in here to compute useful stuff. In this section you'll find a list of useful functions, as well as how to use them, to compute useful numbers for your Yomi games.

Many of the functions require you to know how the characters are numbered. The following table tells you the number for each character.

In [96]:
pd.DataFrame(np.array([i for i in range(20)]), index=names, columns=['Character number'])

,Character number
Grave,0
Jaina,1
Midori,2
Setsuki,3
Rook,4
Degrey,5
Valerie,6
Geiger,7
Lum,8
Argagarg,9


###Counterpick
If it's your turn to counterpick, but you don't know who to choose, this function will give you the most optimal choice!

The function `counterpick(n, m, i)` takes as inputs the number `n` of wins you need to win the match, the number `m` of wins the opponent needs to win the match and the character number `i`.

For example, in the following case: you need 2 games to win, the opponent needs 1 and they just won using Argagarg, character number 9.

The function will tell you to play Troq. Note that this would have been counterintuitive if you tried to choose this according to the Bo1 MUC, since Gloria has a better MU against Argagarg than Troq does. However, Gloria has a $37.81\%$ probability to lose against Zane, so you'd be giving your opponent a very advantaged counterpick in the next game.

Picking Troq, instead, gives the opponent a $40.94\%$ MU using Geiger.

In [102]:
counter = counterpick(2, 1, 9)
print(f"If you pick {names[int(counter[0])]} you will have a {round(counter[1]*100, 2)}% probability of winning")

If you pick Troq you will have a 25.33% probability of winning


###Suboptimal Counterpicks
It's not a given that, for every state, you know how to play the best possible counterpick as good as you need to win a game. You might be interested in seeing other suboptimal counterpicks.

The following script tells you the counterpicks, from best to worst, to a specific state. You can just pick the best one among those that you can play and stick with that.

Mind that this isn't mathematically correct, since the computations for the optimal counterpick assume that you know how to play every character equally good. However, it's a good first approximation.

To use the following script, you only need to change `i` to the index of the character your opponent is using, `n` to the number of wins you need to win the match and `m` to the number of wins your opponent needs to win the match.

In [117]:
i=5
n=3
m=2

MUs = np.zeros(20)
for char in range(20):
  MUs[char] = MUC[char, i]*(1-counterpick(m, n-1, char)[1]) + (1-MUC[char, i])*counterpick(n, m-1, i)[1]

dfToPercent(pd.DataFrame(MUs.T, index=names, columns=["Winning %"]).sort_values(by="Winning %", ascending=False))

,Winning %
Troq,28.71%
Zane,28.61%
Setsuki,28.48%
Persephone,27.7%
Degrey,27.55%
Quince,26.18%
Geiger,26.12%
Rook,25.58%
Menelker,25.42%
Jaina,25.2%


###Optimal counter to a strategy
If your opponent uses a different strategy from the one in the historical data, you can prepare by countering him harder. For example, if you know your opponent uses Rook half of the times and Onimaru the other half, you can build a strategy vector with $0.5$ on both Onimaru's and Rook's index and use the result from the following script as a guideline to see the most likely characters to win the Bo5.

In [119]:
s = np.zeros(20)
s[4] = 0.5
s[11] = 0.5

dfToPercent(pd.DataFrame(np.dot(MUC5, s), index=names, columns=["Winning %"]).sort_values(by="Winning %", ascending=False))

,Winning %
Degrey,54.86%
Lum,53.23%
Geiger,52.8%
Argagarg,52.31%
Bal-Bas-Beta,52.09%
Persephone,52.06%
Grave,52.04%
Quince,51.22%
Valerie,51.09%
Gloria,50.75%


###Lum's Lucky Lottery
Lum's Lucky Lottery is a tournament played in a Bo7 format in which both players pick a random character in the first game.

After that, the losing player can either use a random character or make the opponent switch to a random character (this is called reroll).

Choosing whether to reroll your character or your opponent's has a lot of decision-making to it. For example: you lose with Setsuki against DeGrey. What do you do? Do you reroll your character? Do you make your opponent's reroll?

One approach could be simply seeing which of the two scenarios gives you the largest lead in the next game. This can be done with the following script, by simply changing `yourChar` to the index of your character and `opponentChar` to the index of your opponent's character.

In [134]:
yourChar = 3
opponentChar = 5

def rerollProb(i, j, s):
  if s==1:
    return np.mean(MUC[yourChar, :])
  if s==0:
    return np.mean((1-MUC[:, opponentChar]))
    
print(f"By making the opponent reroll you get a {round(rerollProb(yourChar, opponentChar, 1)*100, 2)}% probability of winning the next game")
print(f"By rerolling your character you get a {round(rerollProb(yourChar, opponentChar, 0)*100, 2)}% probability of winning the next game")


By making the opponent reroll you get a 51.5% probability of winning the next game
By rerolling your character you get a 53.27% probability of winning the next game


##Conclusion
This document is a live document. It's expected to be edited in the future, and it is likely to have more than one error inside. If you can point any error out, just DM on [Sirlin Games](https://sirlingames.discoursehosting.net/u/tonyhole/summary) and I'll fix it ASAP.

As per its practical uses, Yomi is not poker. It's not as popular nor as important to win at Yomi as it might be for poker.

I don't expect whatever's written in this document to change the future meta of Yomi, nor to have any impact on society or to make me millionaire.

Understanding and using this document well ***MAY*** give you an edge in Yomi, but there's so much math, and so many assumptions (and even only the assumption of the MUC being approximately correct) that, in the end, going with your guts will work more or less the same.

As for why I wrote this: this is, basically, the culmination of my hobbies. It's a way to merge my passion for math, programming, card games and fighting games. I also wrote all of this while listening to music and procrastinating the actual work I should get done. Both listening to music and procrastinating are hobbies to me, so yay!

And, finally, this was a way to get some practice with linear programming and optimization, as well as some vectorization practice and other programming-related stuff.